The same Jupyter notebook can be executed in a Google Colab from the following link:

[Open in Colab](https://colab.research.google.com/drive/15fr8iI1xKCY2uMjqT4YHu_FAmTpiCzQG?usp=sharing)

## Q1
How has the number of mass shootings evolved in the big US regions between two concrete years? For this, we need you to aggregate the data in the 5 regions (Southeast, Northeast, Midwest, Northwest, and Southwest), and let the user select the first and last year of the comparison. Same for states, both views coordinated.

In [1]:
import altair as alt
import pandas as pd
import json
import itertools
import numpy as np

In [2]:
data_shootings = pd.read_csv('data/gun_violence_processed.csv')
state_data = pd.read_csv('data/state_data.csv').rename(columns={'state': 'State'})

# Add extra state information to the shootings data
data_shootings = data_shootings.merge(state_data, on='State')

data_shootings['Incident Date'] = pd.to_datetime(data_shootings['Incident Date'])
data_shootings['Year'] = data_shootings['Incident Date'].dt.year
data_shootings['Month'] = data_shootings['Incident Date'].dt.month

data_shootings.loc[:,'count'] = 1  # Add a count column for aggregation


state_df = data_shootings.groupby(['State', 'Year']).agg({
    'region': 'first',
    'Victims Killed': 'sum',
    'Victims Injured': 'sum',
    'Population_per_state_2023': 'first',
    'count': 'sum',
    'FIPS': 'first'
}).rename(columns={
    'region': 'Region',
    'Victims Killed': 'Total Victims Killed',
    'Victims Injured': 'Total Victims Injured',
    'Population_per_state_2023': 'Population',
    'count': 'Total Incidents'
}).reset_index()

# fill empty years with 0 incidents
all_years = state_df['Year'].unique()
all_states = state_df['State'].unique()

for state in all_states:
    pop = state_df[state_df['State'] == state]['Population'].values[0]
    region = state_df[state_df['State'] == state]['Region'].values[0]
    fips = state_df[state_df['State'] == state]['FIPS'].values[0]
    current_years = state_df[state_df['State'] == state]['Year'].values
    for year in all_years:
        if year not in current_years:
            state_df = pd.concat([state_df, pd.DataFrame([{
                'State': state,
                'Year': year,
                'Region': region,
                'Total Victims Killed': 0,
                'Total Victims Injured': 0,
                'Population': pop,
                'Total Incidents': 0,
                'FIPS': fips
            }])], ignore_index=True)
        
# dataset used for the choropleth map
state_statistics_df = state_df.groupby(['State']).agg({
    'Region': 'first',
    'Total Victims Killed': 'sum',
    'Total Victims Injured': 'sum',
    'Population': 'first',
    'Total Incidents': 'sum',
    'FIPS': 'first'
}).reset_index()


region_df = state_df.groupby(['Region', 'Year']).agg({
    'Total Victims Killed': 'sum',
    'Total Victims Injured': 'sum',
    'Population': 'sum',
    'Total Incidents': 'sum'
}).reset_index()

state_df["Type"] = "State"
region_df["Type"] = "Region"

state_df.rename(columns={"State": "Area"}, inplace=True)
region_df.rename(columns={"Region": "Area"}, inplace=True)


area_df = pd.concat([state_df, region_df])

# add average evolution in the US
us_df = state_df.groupby(['Year']).agg({
    'Total Victims Killed': 'sum',
    'Total Victims Injured': 'sum',
    'Population': 'sum',
    'Total Incidents': 'sum'
}).reset_index()

us_df['Area'] = 'United States'
us_df['Type'] = 'Country'

area_df = pd.concat([area_df, us_df]).reset_index(drop=True)

area_df['Count per 100k'] = area_df['Total Incidents'] / area_df['Population'] * 100_000


state_statistics_df['StateFIPS'] = state_statistics_df['FIPS'].astype(str).str[:-3].astype(int)

state_statistics_df['Count per 100k'] = state_statistics_df['Total Incidents'] / state_statistics_df['Population'] * 100_000

area_df

Area  Year     Region  Total Victims Killed  \
0          Alabama  2014  Southeast                     1   
1          Alabama  2015  Southeast                     4   
2          Alabama  2016  Southeast                    15   
3          Alabama  2017  Southeast                     6   
4          Alabama  2018  Southeast                    11   
..             ...   ...        ...                   ...   
565  United States  2019        NaN                   428   
566  United States  2020        NaN                   495   
567  United States  2021        NaN                   668   
568  United States  2022        NaN                   642   
569  United States  2023        NaN                   716   

     Total Victims Injured  Population  Total Incidents    FIPS     Type  \
0                        9     5108468                2  1039.0    State   
1                       13     5108468                4  1097.0    State   
2                       53     5108468               15  1045.0    State   
3                       38     5108468                7  1109.0    State   
4                       53     5108468               14  1091.0    State   
..                     ...         ...              ...     ...      ...   
565                   1690   334914895              414     NaN  Country   
566                   2526   334914895              611     NaN  Country   
567                   2784   334914895              689     NaN  Country   
568                   2647   334914895              644     NaN  Country   
569                   2689   334914895              656     NaN  Country   

     Count per 100k  
0          0.039151  
1          0.078301  
2          0.293630  
3          0.137027  
4          0.274055  
..              ...  
565        0.123613  
566        0.182434  
567        0.205724  
568        0.192288  
569        0.195871  

[570 rows x 10 columns]

Load the preprocessed electoral data

In [3]:
area_electoral_df = pd.read_csv('data/area_electoral_df.csv')

area_electoral_df = area_electoral_df.merge(area_df[['Area', 'Year', 'Count per 100k']], on=['Area', 'Year'])

# keep only the party with the highest electoral votes in each area and year
area_electoral_highest_df = area_electoral_df.groupby(['Year', 'Area']).apply(
    lambda x: x.loc[x['Electoral votes'].idxmax()]
).reset_index(drop=True)

us_party_in_power = area_electoral_highest_df[area_electoral_highest_df['Area'] == 'United States'].reset_index(drop=True)

area_electoral_highest_df

C:\Users\Joel\AppData\Local\Temp\ipykernel_1144\2603903764.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  area_electoral_highest_df = area_electoral_df.groupby(['Year', 'Area']).apply(


Year           Area       Party  Electoral votes  Count per 100k
0    2014        Alabama  Republican                9        0.039151
1    2014         Alaska  Republican                3        0.136350
2    2014        Arizona  Republican               11        0.013457
3    2014       Arkansas  Republican                6        0.065195
4    2014     California    Democrat               55        0.105222
..    ...            ...         ...              ...             ...
565  2023       Virginia    Democrat               13        0.080315
566  2023     Washington    Democrat               12        0.191991
567  2023  West Virginia  Republican                5        0.000000
568  2023      Wisconsin    Democrat               10        0.152260
569  2023        Wyoming  Republican                3        0.000000

[570 rows x 5 columns]

In [4]:
from vega_datasets import data as vega_data

# ======== Chart dimensions ========
MAP_DIM = (300, 300)
LINE_CHART_DIM = (400, 320)
TITLE_HEIGHT = 20
SLIDER_HEIGHT = 20

# Categorical palettes for the regions and areas, both colorblind safe
region_palette = alt.Scale(
    domain=['Southeast', 'Northeast', 'Midwest', 'Northwest', 'Southwest'],
    range=['#648fff','#785ef0','#dc267f','#fe6100','#ffb000']
    )
area_palette_range = ['#2E2585', '#009E73', '#E69F00']

# Parameter definition
state_selection = alt.selection_point(fields=['State'], name='SelectState', empty='none')
region_selection = alt.selection_point(fields=['Region'], name='SelectRegion', empty='none')
year_selection = alt.selection_interval(fields=['Year'], name='SelectYear', encodings=['x'], translate=False)


state_map = alt.Chart(alt.topo_feature(vega_data.us_10m.url, feature='states')).mark_geoshape(
    stroke='white',
    strokeWidth=1
).encode(
    color=alt.Color(
        'Region:N',
        scale=region_palette,
        legend=alt.Legend(orient='top', offset=-TITLE_HEIGHT)
    ),
    opacity={
        "condition": [
            {"test": "datum.State == SelectState.State", "value": 1},
            {"test": "datum.Region == SelectRegion.Region", "value": 0.8}
        ],
        "value": 0.6
    },
    tooltip=['State:N', 'Region:N']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_statistics_df, 'StateFIPS', ['State', 'Region'])
).add_params(
    state_selection, region_selection
).properties(
    width=MAP_DIM[0],
    height=MAP_DIM[1]
).project(
    type='albersUsa'
)

year_selector = alt.Chart(us_party_in_power).mark_rect().encode(
    x=alt.X('Year:O', title='', axis=None),
    color=alt.Color(
        'Party:N',
        title='Party in power',
        scale=alt.Scale(
            domain=['Democrat', 'Republican'],
            range=['blue', 'red']
        ),
        legend=alt.Legend(orient='bottom')
    ),
    opacity=alt.condition(
        year_selection,
        alt.value(0.8),
        alt.value(0.4)
    ),
).properties(
    width=MAP_DIM[0],
    height=SLIDER_HEIGHT
).add_params(
    year_selection
)

year_text = alt.Chart(us_party_in_power).mark_text(
    align='center',
    baseline='middle',
    dy=1,
    fontWeight='bold',
).encode(
    x=alt.X('Year:O'),
    text=alt.Text('Year:O')
)

year_selector += year_text

# generate a dataframe with all the possible titles to make the title dynamic
title_df = state_data[["State", "region"]].copy()
title_df["title"] = title_df.apply(
    lambda x: f"Gun Violence evolution in {x['State']} and {x['region']}",
    axis=1
)

# add a base title where state and region are not selected
title_df = pd.concat([title_df, pd.DataFrame([{
    'State': pd.NA,
    'region': pd.NA,
    'title': 'Gun Violence evolution in the United States'
}])])

line_chart_title = alt.Chart(title_df).mark_text(
    align='left',
    fontSize=13,
    fontWeight='bold',
).encode(
    text=alt.condition(
        "datum.State == SelectState.State && datum.region == SelectRegion.Region",
        'title:N',
        alt.value('')
    )
).properties(
    height=TITLE_HEIGHT
)

line_chart = alt.Chart(area_df).mark_line(point=True).encode(
    x=alt.X('Year:O', title='', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Count per 100k:Q'),
    color=alt.Color(
        'Area:N',
        scale=alt.Scale(range=area_palette_range),
        sort=alt.Sort(field='Type', order='ascending')
    ),
    tooltip=['Year:O', 'Count per 100k:Q']
).transform_filter(
    "datum.Area == SelectState.State || datum.Area == SelectRegion.Region || datum.Area == 'United States'"
).transform_filter(
    year_selection
).properties(
    width=LINE_CHART_DIM[0],
    height=LINE_CHART_DIM[1] - TITLE_HEIGHT
)


year_selector = alt.Chart(us_party_in_power).mark_rect().encode(
    x=alt.X('Year:O', title='', axis=None),
    color=alt.Color(
        'Party:N',
        title='Party in power',
        scale=alt.Scale(
            domain=['Democrat', 'Republican'],
            range=['blue', 'red']
        ),
        legend=alt.Legend(orient='bottom')
    ),
    opacity=alt.condition(
        year_selection,
        alt.value(0.8),
        alt.value(0.4)
    ),
).properties(
    width=MAP_DIM[0],
    height=15
).add_params(
    year_selection
)

year_text = alt.Chart(us_party_in_power).mark_text(
    align='center',
    baseline='middle',
    dy=1,
    fontWeight='bold',
).encode(
    x=alt.X('Year:O'),
    text=alt.Text('Year:O')
)

year_selector += year_text

# create an empty chart to make the final plot aligned
empty_chart = alt.Chart(height=TITLE_HEIGHT, width=MAP_DIM[0]).mark_circle(opacity=0)

aligned_state_map = empty_chart & state_map

def merged_Q1_chart(state_map, year_selector, line_chart_title, line_chart):
    return alt.hconcat(
        alt.vconcat(
            state_map,
            year_selector
        ).resolve_scale(
            color='independent',
        ),
        alt.vconcat(
            line_chart_title,
            line_chart
        ),
    ).resolve_scale(
        color='independent'
    ).configure_title(
        fontSize=16
    )

plot1 = merged_Q1_chart(aligned_state_map, year_selector, line_chart_title, line_chart)
plot1

alt.HConcatChart(...)

In [5]:
# Add labels to the line chart to show the different states
labels = alt.Chart(area_df).mark_text(align='left', dx=5, dy=-5).encode(
    x=alt.X('Year:O', title="", aggregate='max'),
    y=alt.Y('Count per 100k:Q', title="Count per 100k", aggregate={'argmax': 'Year'}),
    text=alt.Text('Area:N'),
    color=alt.Color('Area:N'),
).transform_filter(
    "datum.Area == SelectState.State || datum.Area == SelectRegion.Region || datum.Area == 'United States'"
).transform_filter(
    year_selection
)

line_chart_labeled = line_chart.copy()

# remove the legend and padding to get a cleaner chart
line_chart_labeled.encoding.color.legend = None
line_chart_labeled.encoding.x.scale = alt.Scale(padding=0)

line_chart_labeled += labels


plot1 = merged_Q1_chart(aligned_state_map, year_selector, line_chart_title, line_chart_labeled)
plot1

alt.HConcatChart(...)

For some of the states, the labels overlap, therefore we will keep the legend as before

In [6]:
# Background color in terms of the political party in power
RD_background = alt.Chart(us_party_in_power).mark_rect(opacity=0.2).encode(
    x=alt.X('Year:O', title='', scale=alt.Scale(padding=0)),
    color=alt.Color(
        'Party:N',
        scale=alt.Scale(domain=['Democrat', 'Republican'], range=['blue', 'red']),
        legend=None
    )
).transform_filter(
    year_selection
)


plot1 = merged_Q1_chart(aligned_state_map, year_selector, line_chart_title,
                        (line_chart + RD_background).resolve_scale(color='independent'))
plot1

alt.HConcatChart(...)

Too much clutter. Poor data to ink ratio.

In [7]:
# Add colored points to the line chart in terms of the winning party in each area
party_points_chart = alt.Chart(area_electoral_highest_df).mark_point(filled=True).encode(
    x=alt.X('Year:O', title='', axis=alt.Axis(labelAngle=0), scale=alt.Scale(padding=0)),
    y=alt.Y('Count per 100k:Q'),
    color=alt.Color(
        'Party:N',
        title=['Winning party', 'in the area'],
        scale=alt.Scale(domain=['Democrat', 'Republican'], range=['blue', 'red']),
    ),
    size=alt.value(40),
    opacity=alt.value(0.8),
    tooltip=['Year:O', alt.Tooltip('Count per 100k:Q',format='.2f'), alt.Tooltip('Party:N',title='Winning party'), 'Electoral votes:Q']
).transform_filter(
    "datum.Area == SelectState.State || datum.Area == SelectRegion.Region || datum.Area == 'United States'"
).transform_filter(
    year_selection
).properties(
    width=LINE_CHART_DIM[0],
    height=LINE_CHART_DIM[1] - TITLE_HEIGHT
)


plot1 = merged_Q1_chart(aligned_state_map, year_selector, line_chart_title,
                        (line_chart.mark_line() + party_points_chart).resolve_scale(color='independent'))
plot1

alt.HConcatChart(...)

In [8]:
# Define the fixed y-axis scale
max_state_count = area_df[area_df['Area'] != 'District of Columbia']['Count per 100k'].max()
fixed_y_scale = alt.Scale(domain=[0, max_state_count])

line_chart_scale_fix = line_chart.mark_line().copy()
party_points_chart_scale_fix = party_points_chart.copy()

line_chart_scale_fix.encoding.y.scale = fixed_y_scale
party_points_chart_scale_fix.encoding.y.scale = fixed_y_scale

plot1 = merged_Q1_chart(aligned_state_map, year_selector, line_chart_title,
                        (line_chart_scale_fix + party_points_chart_scale_fix).resolve_scale(color='independent',strokeDash='independent'))
plot1

alt.HConcatChart(...)

Too much clutter. Poor usage of space.

In [9]:
line_chart_stroke = line_chart.mark_line().copy()
# add a stroke dash depending on the area
line_chart_stroke.encoding.strokeDash = alt.StrokeDash('Area:N', sort=alt.Sort(field='Type', order='ascending'))

plot1_line_chart = (line_chart_stroke + party_points_chart).resolve_scale(color='independent',strokeDash='independent')

plot1 = merged_Q1_chart(aligned_state_map, year_selector, line_chart_title, plot1_line_chart)
plot1

alt.HConcatChart(...)

Description:
The visualization system enables analysis of mass shooting incidents across different geographic levels (state, region, country) with temporal filtering. The default view shows incidents per 100,000 people (normalized to allow comparisons) nationwide, serving as a baseline for comparison with state/regional trends. The y-axis scale adjusts dynamically to optimize space usage while maintaining clarity.
Political context is integrated through two features: the year slider showing the presidential administration for each year, and party affiliation indicators for selected states over time (defaulting to presidential party for national data). This allows users to explore potential correlations between political factors and shooting incidents.
The slider enables selecting a year range for the line plot.
The line plot employs dual encoding for geographic granularity, using both line styles and colors to ensure accessibility for colorblind users. Region-specific colors follow IBM's colorblind-safe palette. A dynamic title updates to reflect the selected region, minimizing unnecessary eye movement and maintaining analytical context.
This design enables users to investigate questions like "How did mass shootings evolve in specific regions during certain administrations?", "Is there a correlation between voting patterns and gun incidents?" and "how did gun incidents evolve in specific counties (states) with respect to the state (country)?".

## Q2
Given a concrete year, how has the number of mass shooting per citizen grown or 
decreased across the different regions in the US compared to the first year of sampled data?

In [10]:
region_df['Count per 100k'] = region_df['Total Incidents'] / region_df['Population'] * 100_000

SLOPE_CHART_DIM = (300, 300)
BAR_CHART_DIM = (300, 300)

TITLE_HEIGHT = 20
SLIDER_HEIGHT = 20

base_year = all_years[0]

region_df.rename(columns={'Area': 'Region'}, inplace=True)

region_df = region_df.sort_values(by=['Region', 'Year'])

# Calculate percentage change with respect to the base year
region_df['Percentage Change'] = region_df.groupby('Region')['Count per 100k'].apply(lambda x: (x - x.iloc[0]) / x.iloc[0] * 100).reset_index(level=0, drop=True)

# Create a selection slider for the year with ordinal values
year_select = alt.selection_point(fields=['Year'], value=[{'Year': all_years[-1]}], name='YearSelect')


title_df = pd.DataFrame({'Year': all_years})
title_df["title"] = title_df.apply(
    lambda x: f"Gun Incidents per 100k citizens: {base_year} vs {x['Year']}",
    axis=1
)

slope_chart_title = alt.Chart(title_df).mark_text(
    align='left',
    fontSize=13,
    fontWeight='bold',
).encode(
    text=alt.condition(
        "datum.Year == YearSelect.Year",
        'title:N',
        alt.value('')
    )
).properties(
    height=TITLE_HEIGHT
)

slope_chart = alt.Chart(region_df).mark_line(point=True).encode(
    x=alt.X('Year:O', title=''),
    y='Count per 100k:Q',
    color=alt.Color('Region:N', scale=region_palette, legend=None),
    detail='Region:N',
    tooltip=['Region:N', alt.Tooltip('Count per 100k:Q', format='.2f')]
).transform_filter(
    f"datum.Year == YearSelect.Year || datum.Year == {base_year}"
).properties(
    width=SLOPE_CHART_DIM[0],
    height=SLOPE_CHART_DIM[1]-TITLE_HEIGHT,
)

title_df = pd.DataFrame({'Year': all_years})
title_df["title"] = title_df.apply(
    lambda x: f"Percentage Change in Gun Incidents: {base_year} to {x['Year']}",
    axis=1
)

bar_plot_title = alt.Chart(title_df).mark_text(
    align='left',
    fontSize=13,
    fontWeight='bold',
).encode(
    text=alt.condition(
        "datum.Year == YearSelect.Year",
        'title:N',
        alt.value('')
    )
).properties(
    height=TITLE_HEIGHT
)

# Bar plot to display the percentage change
bar_plot = alt.Chart(region_df).mark_bar().encode(
    x=alt.X('Region:N', title=''),
    y='Percentage Change:Q',
    color=alt.Color('Region:N', scale=region_palette),
    tooltip=['Region:N', alt.Tooltip('Percentage Change:Q', format='.2f')]
).transform_filter(
    year_select
).properties(
    width=BAR_CHART_DIM[0],
    height=BAR_CHART_DIM[1]-TITLE_HEIGHT,
)

year_data = pd.DataFrame({
    'Year': [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
})

year_selector_disc = alt.Chart(year_data).mark_rect().encode(
    x=alt.X('Year:O', title='', axis=None),
    opacity=alt.condition(year_select, alt.value(1.0), alt.value(0.6)),
    color=alt.value('grey')
).properties(
    width=SLOPE_CHART_DIM[0],
    height=SLIDER_HEIGHT
).add_params(
    year_select
)

year_text = alt.Chart(year_data).mark_text(
    align='center',
    baseline='middle',
    dy=1,
    fontWeight='bold',
).encode(
    x=alt.X('Year:O'),
    text=alt.Text('Year:O')
).properties(
    width=SLOPE_CHART_DIM[0],
    height=SLIDER_HEIGHT
)

plot2 = alt.hconcat(
    alt.vconcat(slope_chart_title,slope_chart,year_selector_disc + year_text),alt.vconcat(bar_plot_title,bar_plot)
).configure_axisX(
    labelAngle=0
).resolve_scale(
    x='independent'
)

plot2

alt.HConcatChart(...)

Try having fixed y-scale.

In [11]:
# Define the fixed y-axis scales
fixed_y_scale_slope = alt.Scale(domain=[0, region_df['Count per 100k'].max()+0.01])
fixed_y_scale = alt.Scale(domain=[region_df['Percentage Change'].min(), region_df['Percentage Change'].max()])

# Update the slope chart and bar plot with a fixed y-axis
slope_chart_fixed_scale = slope_chart.copy()
slope_chart_fixed_scale.encoding.y.scale = fixed_y_scale_slope

bar_plot_fixed_scale = bar_plot.copy()
bar_plot_fixed_scale.encoding.y.scale = fixed_y_scale

combined_chart = alt.hconcat(
    alt.vconcat(slope_chart_title, slope_chart_fixed_scale, year_selector_disc + year_text),
    alt.vconcat(bar_plot_title, bar_plot_fixed_scale)
).configure_axisX(
    labelAngle=0
).resolve_scale(
    x='independent'
)

combined_chart

alt.HConcatChart(...)

Poor usage of space, poor data-to-ink ratio, more clutter. Does not bring much value in terms of the tasks to solve.

In [12]:
plot2

alt.HConcatChart(...)

Description:
The visualization system analyzes gun violence trends using two complementary charts that compare data between an initial reference year and a selected year across different regions. The first chart is a slope chart showing absolute values of incidents per 100,000 citizens, while the second is a bar chart displaying percentage changes between the two years.

The year selection widget defaults to the last year (closest to the time of visualization).

The dual-chart approach enables two distinct types of analysis: absolute comparison of violence rates between regions (slope chart) and relative changes in violence rates (bar chart). For instance, while the Southeast or Midwest might show the highest absolute increase in incidents per capita in 2019, the Southwest demonstrated the largest relative increase at 100%.

Both charts utilize dynamic y-axes, made effective by the slope chart's fixed starting point and the bar chart's focus on relative changes from the baseline year. Interactive tooltips provide precise values, compensating for the dynamic scales where users cannot rely on memorized reference points with the fixed scale.

The coordinated views allow simultaneous examination of absolute and relative changes, providing users with a comprehensive understanding of gun violence evolution across regions.

## Q3
For the visualization in Q1, we would like you to be able to select a state, and show the
detailed information on the counties of the state.

In [13]:
state_abbr_to_name = json.load(open('data/state_abbreviations.json'))
fips_to_county_df = pd.read_csv('data/State and County FIPS Codes.csv')

data_shootings.loc[:,'count'] = 1  # Add a count column for aggregation

fips_to_county_df.rename(columns={'fips': 'FIPS', 'name':'county_name'}, inplace=True)

county_statistics_df = data_shootings.groupby('FIPS').agg({
    'Victims Killed': 'sum',
    'Victims Injured': 'sum',
    'Population_per_county_2023': 'first',
    'count': 'sum',
}).rename(columns={
    'count': 'shooting_count',
    'Population_per_county_2023': 'population'
}).reset_index()

# Ensure FIPS codes are strings and padded to 5 digits
county_statistics_df['FIPS'] = county_statistics_df['FIPS'].astype(str).str.zfill(5)
fips_to_county_df['FIPS'] = fips_to_county_df['FIPS'].astype(str).str.zfill(5)

fips_to_county_df['state'] = fips_to_county_df.apply(
    lambda x: x['county_name'] if pd.isnull(x['state']) else state_abbr_to_name.get(x['state'], x['state']), axis=1
)

# Merge with fips_to_county_df to include all counties, using a left join
county_statistics_df = pd.merge(fips_to_county_df[['FIPS', 'county_name','state']], county_statistics_df, on='FIPS', how='left')

county_statistics_df['Count per 100k'] = (county_statistics_df['shooting_count'] / county_statistics_df['population']) * 100_000
county_statistics_df['Deaths per 100k'] = (county_statistics_df['Victims Killed'] / county_statistics_df['population']) * 100_000
county_statistics_df['Injured per 100k'] = (county_statistics_df['Victims Injured'] / county_statistics_df['population']) * 100_000

county_statistics_df['county_name'] = county_statistics_df['county_name'].str.replace(' County', '')

# the counties that were not present in the shootings data are filled with 0
county_statistics_df.fillna(0, inplace=True)

county_statistics_df.rename(columns={'county_name': 'County', 'state': 'State'}, inplace=True)

# keep only the columns we need
county_statistics_df = county_statistics_df[['FIPS', 'State', 'County', 'Count per 100k', 'Deaths per 100k', 'Injured per 100k']]

county_statistics_df

FIPS          State         County  Count per 100k  Deaths per 100k  \
0     00000  UNITED STATES  UNITED STATES        0.000000         0.000000   
1     01000        ALABAMA        ALABAMA        0.000000         0.000000   
2     01001        Alabama        Autauga        3.314441         6.628882   
3     01003        Alabama        Baldwin        0.394466         1.577866   
4     01005        Alabama        Barbour        0.000000         0.000000   
...     ...            ...            ...             ...              ...   
3191  56037        Wyoming     Sweetwater        0.000000         0.000000   
3192  56039        Wyoming          Teton        0.000000         0.000000   
3193  56041        Wyoming          Uinta        0.000000         0.000000   
3194  56043        Wyoming       Washakie        0.000000         0.000000   
3195  56045        Wyoming         Weston        0.000000         0.000000   

      Injured per 100k  
0             0.000000  
1             0.000000  
2             6.628882  
3             0.000000  
4             0.000000  
...                ...  
3191          0.000000  
3192          0.000000  
3193          0.000000  
3194          0.000000  
3195          0.000000  

[3196 rows x 6 columns]

In [14]:
county_df = data_shootings.groupby(['County', 'State', 'Year']).agg({
    'Victims Killed': 'sum',
    'Victims Injured': 'sum',
    'Population_per_county_2023': 'first',
    'count': 'sum'
}).rename(columns={
    'Population_per_county_2023': 'Population',
    'count': 'Total Incidents'
}).reset_index()

county_df['County'] = county_df['County'].str.title()

county_df['Count per 100k'] = county_df['Total Incidents'] / county_df['Population'] * 100_000

# Create a dataframe with all possible combinations of counties and years
all_combinations = county_df[['County', 'State', 'Population']].drop_duplicates().assign(key=1).merge(
    pd.DataFrame(all_years, columns=['Year']).assign(key=1), on='key').drop('key', axis=1)

# Merge with the original dataframe to ensure all combinations are present
county_df = all_combinations.merge(county_df, on=['County', 'State', 'Population', 'Year'], how='left')

county_df.fillna(0, inplace=True)

county_df

County           State  Population  Year  Victims Killed  \
0     Abbeville  South Carolina     24434.0  2014             0.0   
1     Abbeville  South Carolina     24434.0  2015             0.0   
2     Abbeville  South Carolina     24434.0  2016             0.0   
3     Abbeville  South Carolina     24434.0  2017             0.0   
4     Abbeville  South Carolina     24434.0  2018             0.0   
...         ...             ...         ...   ...             ...   
7582       Yuma         Arizona    213221.0  2019             0.0   
7583       Yuma         Arizona    213221.0  2020             0.0   
7584       Yuma         Arizona    213221.0  2021             0.0   
7585       Yuma         Arizona    213221.0  2022             0.0   
7586       Yuma         Arizona    213221.0  2023             2.0   

      Victims Injured  Total Incidents  Count per 100k  
0                 0.0              0.0        0.000000  
1                 0.0              0.0        0.000000  
2                 0.0              0.0        0.000000  
3                 0.0              0.0        0.000000  
4                 0.0              0.0        0.000000  
...               ...              ...             ...  
7582              0.0              0.0        0.000000  
7583              0.0              0.0        0.000000  
7584              0.0              0.0        0.000000  
7585              4.0              1.0        0.468997  
7586             10.0              2.0        0.937994  

[7587 rows x 8 columns]

In [15]:
# keep only the top n counties for each state with the highest count per 100k across all years
def keep_top_n(df, n=10, column='Count per 100k', add_US=True):
    res_df = pd.DataFrame()
    for state, state_df in df.groupby('State'):
        county_count = {}
        for county, county_df in state_df.groupby('County'):
            county_count[county] = county_df[column].sum()

        top_counties = sorted(county_count.items(), key=lambda x: x[1], reverse=True)[:n]
        top_counties = [x[0] for x in top_counties]

        res_df = pd.concat([res_df, state_df[state_df['County'].isin(top_counties)]], ignore_index=True)

    # add the top n counties in the US
    if add_US:
        county_count = {}
        for (state, county), county_df in df.groupby(['State', 'County']):
            county_count[(state, county)] = county_df[column].sum()

        top_counties = sorted(county_count.items(), key=lambda x: x[1], reverse=True)[:n]
        top_counties = [x[0] for x in top_counties]

        selected_counties = pd.DataFrame()
        for state, county in top_counties:
            selected_counties = pd.concat([selected_counties, df[(df['County'] == county) & (df['State'] == state)]], ignore_index=True)
        selected_counties['County'] = selected_counties['County'] + ', ' + selected_counties['State']
        selected_counties['State'] = pd.NA

        res_df = pd.concat([res_df, selected_counties], ignore_index=True)
    
    return res_df.sort_values(['State', 'County', 'Year']).reset_index(drop=True)

In [16]:
STATE_MAP_DIM = (300, 300)
LINE_CHART_DIM = (400, 300)

# number of counties to plot
N = 5

counties_topojson = 'https://cdn.jsdelivr.net/npm/us-atlas@3/counties-10m.json'

county_map = alt.Chart(alt.topo_feature(counties_topojson, 'counties')).mark_geoshape().encode(
    color=alt.Color(
        'Count per 100k:Q',
        scale=alt.Scale(
            scheme='reds',
            domain=[0, county_statistics_df['Count per 100k'].quantile(0.99)]
        ),
        legend=alt.Legend(orient='bottom'),
        title='Mass Shootings per 100k'
    ),
    tooltip=[
        alt.Tooltip('County:N', title='County'),
        alt.Tooltip('State:N', title='State'),
        alt.Tooltip('Count per 100k:Q', format='.2f'),
        alt.Tooltip('Deaths per 100k:Q', format='.2f'),
        alt.Tooltip('Injured per 100k:Q', format='.2f')
    ]
).transform_lookup(
    lookup='id',  # 'id' in us-10m.json corresponds to county FIPS code
    from_=alt.LookupData(county_statistics_df, 'FIPS', ['Count per 100k', 'County', 'State', 'Deaths per 100k', 'Injured per 100k']),
    default=[0]
).transform_filter(
    "datum.State == SelectState.State || SelectState.State == null"
).project(
    type='albersUsa', precision=.707
).properties(
    width=STATE_MAP_DIM[0],
    height=STATE_MAP_DIM[1],
    title=alt.TitleParams(
        text='Number of Mass Shootings by County (2014-2023)',
        fontSize=13,
        fontWeight='bold'
    )
)


county_line_chart = alt.Chart(keep_top_n(county_df, N)).mark_line(point=True).encode(
    x=alt.X('Year:O', title='', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Count per 100k:Q'),
    color=alt.Color(
        'County:N',
        scale=alt.Scale(scheme='category20'),
        sort=alt.Sort(field='Type', order='ascending'),
        legend=alt.Legend(orient='bottom')
    ),
    tooltip=['Year:O', 'Count per 100k:Q']
).transform_filter(
    "datum.State == SelectState.State"
).transform_filter(
    year_selection
).properties(
    width=LINE_CHART_DIM[0],
    height=LINE_CHART_DIM[1] - TITLE_HEIGHT
)


plot3 = alt.hconcat(
    alt.vconcat(
        state_map,
        year_selector
    ).resolve_scale(
        color='independent',
    ),
    county_map,
    county_line_chart
).resolve_scale(
    color='independent'
).configure_title(
    fontSize=16
)

plot3

alt.HConcatChart(...)

In [17]:
BAR_CHART_DIM = (400, 300)

title_df = pd.DataFrame(
    list(itertools.product(np.append(all_states, pd.NA), all_years, all_years)),
    columns=['State', 'init_year', 'end_year']
)

title_df = title_df[title_df['init_year'] <= title_df['end_year']].reset_index(drop=True)

title_df['title'] = title_df.apply(
    lambda x: f"Top {N} counties in {x['State'] if pd.notnull(x['State']) else 'the United States'} with the highest number of mass shootings from {x['init_year']} to {x['end_year']}"
    if x['init_year'] != x['end_year'] else
    f"Top {N} counties in {x['State'] if pd.notnull(x['State']) else 'the United States'} with the highest number of mass shootings in {x['init_year']}",
    axis=1
)

county_histogram_title = alt.Chart(title_df).mark_text(
    align='left',
    fontSize=13,
    fontWeight='bold',
).encode(
    text=alt.condition(
        "datum.State == SelectState.State &&"
        "(isDefined(SelectYear.Year) ?"
        "datum.init_year == SelectYear.Year[0] && datum.end_year == SelectYear.Year[SelectYear.Year.length - 1] :"
        "datum.init_year == 2014 && datum.end_year == 2023)",
        'title:N',
        alt.value('')
    )
).properties(
    height=TITLE_HEIGHT
)

county_histogram = alt.Chart(keep_top_n(county_df, N)).mark_bar().encode(
    x=alt.X('Count per 100k:Q', title='Number of Mass Shootings per 100k'),
    y=alt.Y('County:N', title='', sort='-x'),
    color=alt.Color(
        'Year:O',
        legend=alt.Legend(title='Year', orient='right'),
        scale=alt.Scale(scheme='viridis', domain=all_years)
    ),
    tooltip=[alt.Tooltip('County:N'), 
             alt.Tooltip('Count per 100k:Q', format='.2f'), 
             alt.Tooltip('Year:O')]
).transform_filter(
    "datum.State == SelectState.State"
).transform_filter(
    year_selection
).properties(
    width=BAR_CHART_DIM[0],
    height=BAR_CHART_DIM[1] - TITLE_HEIGHT
)

plot3 = alt.vconcat(
    alt.hconcat(
        alt.vconcat(
            state_map,
            year_selector
        ).resolve_scale(
            color='independent',
        ),
        county_map
    ).resolve_scale(
        color='independent',
    ),
    alt.vconcat(
        county_histogram_title,
        county_histogram
    ).resolve_scale(
        color='independent',
    ),
    center=True
).resolve_scale(
    color='independent'
).configure_title(
    fontSize=16
)

plot3

alt.VConcatChart(...)

Description:

This visualization addresses the task of identifying counties with the highest incident rates and exploring their temporal/spatial distribution, providing users with a detailed understanding of gun violence patterns within a selected state for the selected year range.

We encountered the unique challenge of displaying sporadic gun incident data at the county level for a selected state. Rather than using a potentially cluttered line chart with numerous zero values, the design implements a stacked bar chart to show temporal distributions for counties with significant incident counts.
The stacked bar approach efficiently identifies counties with the highest number of shootings while simultaneously displaying the temporal distribution of incidents within each county. This method effectively handles the sporadic nature of the data while maintaining temporal context.

Complementing the stacked bars, a choropleth map provides geographical context for county-level incident distribution. The map uses a natural scale to preserve extreme values and facilitate identification of high-incident county clusters.

The design employs a colorblind-safe palette, particularly important as color is the sole encoding method for temporal distribution within county bars. While the choropleth map assumes homogeneous distribution within counties, this limitation is deemed acceptable given the data's nature and visualization goals.



## Final visualization

In [18]:
# Remove slope chart legend and bar plot legend since it's the same as the plot 1 selector legend
slope_chart.encoding.color.legend = None
bar_plot.encoding.color.legend = None

county_line_chart.encoding.color.legend.orient = 'right'

reduced_title_df = title_df.copy()
reduced_title_df['title'] = reduced_title_df.apply(
    lambda x: f"Top {N} counties in {x['State'] if pd.notnull(x['State']) else 'the US'} in Gun Incidents ({x['init_year']}-{x['end_year']})",
    axis=1
)

reduced_county_histogram_title = alt.Chart(reduced_title_df).mark_text(
    align='left',
    fontSize=13,
    fontWeight='bold',
).encode(
    text=alt.condition(
        "datum.State == SelectState.State &&"
        "(isDefined(SelectYear.Year) ?"
        "datum.init_year == SelectYear.Year[0] && datum.end_year == SelectYear.Year[SelectYear.Year.length - 1] :"
        "datum.init_year == 2014 && datum.end_year == 2023)",
        'title:N',
        alt.value('')
    )
).properties(
    height=TITLE_HEIGHT
)

In [19]:
final_vis = alt.hconcat(
    alt.vconcat(state_map,year_selector,county_map).resolve_scale(color='independent'),
    alt.vconcat(line_chart_title,plot1_line_chart,reduced_county_histogram_title,county_histogram).resolve_scale(color='independent', strokeDash='independent'),
    alt.vconcat(slope_chart_title, slope_chart, year_selector_disc + year_text, bar_plot_title, bar_plot)).resolve_scale(color = 'independent').configure_axisX(labelAngle = 0).resolve_scale(color = 'independent', x = 'independent')

final_vis.save('final_vis.html')

final_vis

alt.HConcatChart(...)

Description:

The final visualization will be 1000x620, a ratio corresponding to the required display size. It is divided into three vertical sectors of width 300, 400, and 300. For each vertical sector, there are two horizontal sectors of height 320 and 300 for the upper and lower sectors respectively. The sizes of each plot are selected to ensure that the plots are aligned and that selectors are close to the plots they control. 

The first sector will contain geographic information, both with the selector and the county-level map. 

The second sector contains both the lineplot for Q1 and the barplot of Q3, which maintain proximity to the selector and provide an analysis with full granularity of the incident count: county/state/region/country (bottom to top). 

Finally, we displayed the second plot vertically in the last sector, away from the selector as it displays cross-region information and grouped together to investigate and compare regional trends. 

The color palette is consistent across the plots, with the same color representing the same region, and only specified once to avoid redundancy (although this information is present in tooltips to avoid excessive eye movement to the legend on the left, for example in Q2).